### LangChain chatbot with HeatWave GenAI
With the launch of [HeatWave GenAI](https://www.oracle.com/heatwave/genai/), the platform's capabilities now extend to unstructured data, empowering enterprises to harness the potential of Generative AI. HeatWave GenAI provides a unified environment for Generative AI, integrating all stages of the AI pipeline within the database. It features in-database LLMs, in-database embedding generation, and works seamlessly with other in-database functions such as machine learning, analytics, and Lakehouse. Additionally, HeatWave GenAI allows users the flexibility to leverage external LLM services like [OCI Generative AI](https://www.oracle.com/artificial-intelligence/generative-ai/generative-ai-service/).

[LangChain](https://github.com/langchain-ai/langchain) is a framework designed to simplify the development of applications powered by large language models (LLMs). It provides a set of tools and abstractions to build complex workflows, allowing developers to integrate LLMs with external data sources, APIs, and systems.

HeatWave GenAI can be easily integrated with LangChain applications by specifying a [custom LLM](https://python.langchain.com/docs/how_to/custom_llm/) wrapper class and then using it like any other LLM within LangChain. We use this custom MyLLM class to define a simple LangChain chatbot.


In [1]:
import json
from typing import Any, Dict, List, Optional
from pydantic import Field
import mysql.connector
import pandas as pd
from langchain_core.language_models import LLM
from langchain_core.messages import AIMessage, HumanMessage

/Users/ssaagraw/Documents/GitHub/heatwave-ml/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Connect to the HeatWave instance
We create a connection to an active [HeatWave](https://www.oracle.com/mysql/) instance using the [MySQL Connector/Python](https://dev.mysql.com/doc/connector-python/en/). We also define an API to execute a SQL query using a cursor, and the result is returned as a Pandas DataFrame. Modify the below variables to point to your HeatWave instance. On AWS, set USE_BASTION to False. On OCI, please create a tunnel on your machine using the below command by substituting the variable with their respective values.

ssh -o ServerAliveInterval=60 -i BASTION_PKEY -L LOCAL_PORT:DBSYSTEM_IP:DBSYSTEM_PORT BASTION_USER@BASTION_IP

In [ ]:
BASTION_IP = "ip_address"
BASTION_USER = "opc"
BASTION_PKEY = "private_key_file"
DBSYSTEM_IP = "ip_address"
DBSYSTEM_PORT = 3306
DBSYSTEM_USER = "username"
DBSYSTEM_PASSWORD = "password"
DBSYSTEM_SCHEMA = "ml_benchmark"
LOCAL_PORT = 31231
USE_BASTION = True

if USE_BASTION is True:
    DBSYSTEM_IP = "127.0.0.1"
else:
    LOCAL_PORT = DBSYSTEM_PORT

mydb = mysql.connector.connect(
    host=DBSYSTEM_IP,
    port=LOCAL_PORT,
    user=DBSYSTEM_USER,
    password=DBSYSTEM_PASSWORD,
    database=DBSYSTEM_SCHEMA,
    allow_local_infile=True,
    use_pure=True,
    autocommit=True,
)
mycursor = mydb.cursor()


# Helper function to execute SQL queries and return the results as a Pandas DataFrame
def execute_sql(sql: str) -> pd.DataFrame:
    mycursor.execute(sql)
    return pd.DataFrame(mycursor.fetchall(), columns=mycursor.column_names)

### Define a custom class for MySQL LLMs on top of LangChain's LLM interface
The llm object can now be used as a part of any LangChain invocation. Here is an example of using the above LLM as a Chatbot as described in this LangChain tutorial.

In [3]:
class MyLLM(LLM):
    """
    Custom class for HeatWave LLMs
    """

    # HeatWave GenAI LLM to use. Can be an in-HeatWave LLM or OCI Generative AI LLM.
    # For the list of supported LLMs, refer to
    # https://dev.mysql.com/doc/heatwave/en/mys-hw-genai-supported-models.html
    model_id: str = "llama3.2-3b-instruct-v1"

    # Helper functionper function to execute SQL queries and return the results as a Pandas DataFrame
    def _execute_sql(self, sql: str) -> pd.DataFrame:
        mycursor.execute(sql)
        return pd.DataFrame(mycursor.fetchall(), columns=mycursor.column_names)

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> str:
        """Run the LLM on the given input.

        Args:
            prompt: The prompt to generate from.
            stop: Not supported.

            **kwargs: Arbitrary additional keyword arguments. These are usually passed
                to the model provider API call.

        Returns:
            The model output as a string. Actual completions SHOULD NOT include the prompt.
        """
        if stop is not None:
            raise NotImplementedError("stop kwargs are not permitted.")
        output = self._execute_sql(
            f"""SELECT sys.ML_GENERATE("{prompt}", JSON_OBJECT("task", "generation", "model_id", "{self.model_id}"));"""
        )
        return json.loads(output.iat[0, 0])["text"]

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "custom"

### Create a chatbot using MyLLM
We first create an llm object by instantiating the MyLLM class. The llm object can now be used as a part of any LangChain invocation. Here is an example of using the above LLM as a Chatbot as described in this [LangChain tutorial](https://python.langchain.com/docs/tutorials/chatbot/).

In [4]:
llm = MyLLM()
print(
    llm.invoke(
        [
            HumanMessage(content="Hi! I'm Bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
            HumanMessage(content="What's my name?"),
        ]
    )
)

I don't have any information about your name, Bob. You just told me that you're Bob. Would you like to tell me a bit more about yourself or is there something specific I can help you with?


The above response was generated using the llama3.2-3b-instruct-v1 LLM (default in the above custom class) which is running inside HeatWave on CPUs at no additional cost to you. 

As you can see, using HeatWave GenAI with LangChain is simple, and the above custom class should serve most usecases. You can of course extend the above class to add more supported options like temperature, max_tokens or any other supported parameter.

We invite you to try [HeatWave AutoML and GenAI](https://www.oracle.com/heatwave/free/). If you’re new to Oracle Cloud Infrastructure, try Oracle Cloud Free Trial, a free 30-day trial with US$300 in credits.